<a href="https://colab.research.google.com/github/yohanesnuwara/LiDAR/blob/main/1_read_and_visualize_point_cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 01 Read point cloud data, clip with polygon, and visualize

This tutorial will discuss how to read point cloud data in LAZ format, clip the data by polygon, and visualize it using Python in Google Colab. Why Google Colab, is because it is a free notebook service that offers powerful computing resource. 

To configure the environment, start by running the first cell. 

In [2]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:17
🔁 Restarting kernel...


**Please wait few seconds** until the notebook restarts. Then, run the next cell. 

In [1]:
import condacolab
condacolab.check()

!mamba install -q gdal-bin python-gdal python3-gdal
!mamba install -q python-pdal Shapely Fiona
!apt install python3-rtree
!pip install git+git//github.com/geopandas/geopandas.git
!pip install descartes
!mamba install -q ipyleaflet
!pip install -q ipyvolume
!pip install -q shapely

✨🍰✨ Everything looks OK!

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.0.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████

Encountered problems while solving:
  - nothing provides requested gdal-bin
  - nothing p


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.0.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████

  Package                           Version  Build           Channel                   Size
──────────────────────

Import the libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib
import pdal
import fiona
from shapely.geometry import shape, LineString
import ipyvolume.pylab as p3
import json

from google.colab import output
output.enable_custom_widget_manager()

PROJ: proj_create_from_database: Cannot find proj.db


Clone the repository where the data and functions are stored

In [3]:
!git clone https://github.com/yohanesnuwara/LiDAR

Cloning into 'LiDAR'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 50 (delta 17), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (50/50), done.


## 1 - Sanity check

LiDAR point cloud data often comes as a standard format such as LAS or LAZ (compressed). The data that we are going to use is Borneo forest data acquired by NASA Oak Ridge National Laboratory (ORNL) in 2014. You need to make account. Instead, I have prepared the "subset" of data in the repository. The data is: `Polygon_014_utm_50S_3.laz`

Before we do anything, it's better to do quick sanity check e.g. to spot any possible outlier in the elevation. We will use PDAL for processing LiDAR data. The function to check the statistics of data is PDAL Info. Here we check X,Y,Z dimension. The Z dimension appears to have outlier. Check the JSON output, Z has maximum of 660 m. It is not true because the maximum height of forest can reach up to 40 or 50 m. Thus, later we will remove these outliers. 

In [5]:
!pdal info /content/LiDAR/data/Polygon_014_utm_50S_3.laz --dimensions X,Y,Z

{
  "file_size": 21039074,
  "filename": "/content/LiDAR/data/Polygon_014_utm_50S_3.laz",
  "now": "2022-12-25T14:31:48+0000",
  "pdal_version": "2.4.3 (git-version: dcef18)",
  "reader": "readers.las",
  "stats":
  {
    "bbox":
    {
      "native":
      {
        "bbox":
        {
          "maxx": 219433.94,
          "maxy": 9745664.9,
          "maxz": 660.73,
          "minx": 218620.96,
          "miny": 9744713.06,
          "minz": 4.66
        },
        "boundary": { "type": "Polygon", "coordinates": [ [ [ 218620.96, 9744713.060000000521541, 4.66 ], [ 218620.96, 9745664.900000000372529, 4.66 ], [ 219433.94, 9745664.900000000372529, 660.73 ], [ 219433.94, 9744713.060000000521541, 660.73 ], [ 218620.96, 9744713.060000000521541, 4.66 ] ] ] }
      }
    },
    "statistic":
    [
      {
        "average": 219071.1851,
        "count": 5266966,
        "maximum": 219433.94,
        "minimum": 218620.96,
        "name": "X",
        "position": 0,
        "stddev": 171.401639,


## 2 - Read LAZ data

In PDAL, we are making JSON pipeline to read and process LAS/LAZ data. Every time we add new process, we add new registers into the pipeline. For this part, we read LAZ data and output the array.

In [4]:
pipeline = {
  "pipeline": [
    {
      "type": "readers.las",
      "filename": "/content/LiDAR/data/Polygon_014_utm_50S_3.laz"
    }
  ]
}

r = pdal.Pipeline(json.dumps(pipeline))
# pipeline.validate()
r.execute()

arrays = r.arrays
print(arrays)

[array([(219000.  , 9744763.64,  6.85, 30, 2, 2, 0, 0, 0, -17., 0, 110, 98507277.51513389, 0, 0, 0),
       (219000.46, 9744763.37,  6.46, 21, 2, 2, 0, 0, 2, -17., 0, 110, 98507277.51513988, 0, 0, 0),
       (219000.37, 9744763.53,  6.56, 39, 2, 2, 1, 0, 2, -17., 0, 110, 98507277.51743788, 0, 0, 0),
       ...,
       (218999.17, 9745499.87, 15.79,  1, 1, 4, 0, 0, 0,  13., 0, 110, 98507292.96204288, 0, 0, 0),
       (218999.38, 9745499.87, 14.75, 22, 1, 4, 0, 0, 0,  13., 0, 110, 98507292.96204989, 0, 0, 0),
       (218999.96, 9745499.56, 15.68, 36, 1, 1, 0, 0, 0,  13., 0, 110, 98507292.96205688, 0, 0, 0)],
      dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'), ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'), ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('ScanAngleRank', '<f4'), ('UserData', 'u1'), ('PointSourceId', '<u2'), ('GpsTime', '<f8'), ('Red', '<u2'), ('Green', '<u2'), ('Blue', '<u2')])]


## 2 - Remove outlier

In [6]:
pipeline = {
  "pipeline": [
    {
      "type": "readers.las",
      "filename": "/content/LiDAR/data/Polygon_014_utm_50S_3.laz"
    },
    {
      "type": "filters.range",
      "limits": "Z[:50]"
    },
    {
      "type": "writers.las",
      "filename": "/content/LiDAR/data/Polygon_014_utm_50S_3_OutlierRem.laz"
    }
  ]
}

r = pdal.Pipeline(json.dumps(pipeline))
# pipeline.validate()
r.execute()

arrays = r.arrays
print(arrays)

[array([(219000.  , 9744763.64,  6.85, 30, 2, 2, 0, 0, 0, -17., 0, 110, 98507277.51513389, 0, 0, 0),
       (219000.46, 9744763.37,  6.46, 21, 2, 2, 0, 0, 2, -17., 0, 110, 98507277.51513988, 0, 0, 0),
       (219000.37, 9744763.53,  6.56, 39, 2, 2, 1, 0, 2, -17., 0, 110, 98507277.51743788, 0, 0, 0),
       ...,
       (218999.17, 9745499.87, 15.79,  1, 1, 4, 0, 0, 0,  13., 0, 110, 98507292.96204288, 0, 0, 0),
       (218999.38, 9745499.87, 14.75, 22, 1, 4, 0, 0, 0,  13., 0, 110, 98507292.96204989, 0, 0, 0),
       (218999.96, 9745499.56, 15.68, 36, 1, 1, 0, 0, 0,  13., 0, 110, 98507292.96205688, 0, 0, 0)],
      dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'), ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'), ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('ScanAngleRank', '<f4'), ('UserData', 'u1'), ('PointSourceId', '<u2'), ('GpsTime', '<f8'), ('Red', '<u2'), ('Green', '<u2'), ('Blue', '<u2')])]


In [7]:
crop = fiona.open("/content/LiDAR/data/Polygon_014_utm_50S_3.shp")
b = crop.bounds
collection = [ shape(item['geometry']) for item in crop ][0]
print(collection)

POLYGON ((219045.75029658648 9745597.231219921, 219081.97088975937 9745585.70648573, 219098.98359261328 9745564.852204813, 219097.88599888078 9745541.253939563, 219084.1660772244 9745526.436424173, 219060.01901510914 9745525.887627307, 219037.51834359264 9745536.863564633, 219029.2863905988 9745556.620251818, 219026.54240626752 9745578.023329603, 219032.57917179633 9745591.743251259, 219045.75029658648 9745597.231219921))


In [8]:
pipeline = {
    'pipeline': [
        {
            "type": "readers.las",
            "filename": "/content/LiDAR/data/Polygon_014_utm_50S_3_OutlierRem.laz"
        },    
        {   
            "type":"filters.crop",
            'polygon':collection.wkt
        },       
        {
            "type": "writers.las",
            "filename": "/content/LiDAR/data/Clipped_Polygon_014_utm_50S_3.laz"
        }
    ]
}

r = pdal.Pipeline(json.dumps(pipeline))
# pipeline.validate()
r.execute()

arrays = r.arrays
print(arrays)

[array([(219060.  , 9745526.17,  9.82,  1, 4, 4, 1, 0, 0,  9., 0, 110, 98507294.02690887, 0, 0, 0),
       (219058.9 , 9745526.79,  7.55,  1, 2, 2, 1, 0, 0,  9., 0, 110, 98507294.02692288, 0, 0, 0),
       (219052.09, 9745529.85,  7.32,  9, 4, 4, 0, 0, 2, 10., 0, 110, 98507294.03189088, 0, 0, 0),
       ...,
       (219081.15, 9745585.72, 21.19, 12, 1, 3, 0, 0, 0, 10., 0, 110, 98507295.70475888, 0, 0, 0),
       (219081.61, 9745585.5 , 21.6 , 17, 1, 3, 0, 0, 0, 10., 0, 110, 98507295.70476489, 0, 0, 0),
       (219082.04, 9745585.32, 21.83, 37, 1, 2, 0, 0, 0, 10., 0, 110, 98507295.70477188, 0, 0, 0)],
      dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'), ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'), ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('ScanAngleRank', '<f4'), ('UserData', 'u1'), ('PointSourceId', '<u2'), ('GpsTime', '<f8'), ('Red', '<u2'), ('Green', '<u2'), ('Blue', '<u2')])]


In [9]:
# Load Pipeline output in python objects
arr = r.arrays[0]
description = arr.dtype.descr
cols = [col for col, __ in description]
df = pd.DataFrame({col: arr[col] for col in cols})
df['X_0'] = df['X']
df['Y_0'] = df['Y']
df['Z_0'] = df['Z']
df['X'] = df['X'] - df['X_0'].min()
df['Y'] = df['Y'] - df['Y_0'].min()
df['Z'] = df['Z'] - df['Z_0'].min()

df

,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,UserData,PointSourceId,GpsTime,Red,Green,Blue,X_0,Y_0,Z_0
0,33.40,0.23,2.84,1,4,4,1,0,0,9.0,0,110,9.850729e+07,0,0,0,219060.00,9745526.17,9.82
1,32.30,0.85,0.57,1,2,2,1,0,0,9.0,0,110,9.850729e+07,0,0,0,219058.90,9745526.79,7.55
2,25.49,3.91,0.34,9,4,4,0,0,2,10.0,0,110,9.850729e+07,0,0,0,219052.09,9745529.85,7.32
3,25.91,3.73,0.41,6,4,4,0,0,2,10.0,0,110,9.850729e+07,0,0,0,219052.51,9745529.67,7.39
4,32.43,1.06,0.56,2,4,4,0,0,0,9.0,0,110,9.850729e+07,0,0,0,219059.03,9745527.00,7.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73389,53.46,60.15,15.08,35,1,3,0,0,0,10.0,0,110,9.850730e+07,0,0,0,219080.06,9745586.09,22.06
73390,53.83,60.02,14.83,31,1,2,0,0,0,10.0,0,110,9.850730e+07,0,0,0,219080.43,9745585.96,21.81
73391,54.55,59.78,14.21,12,1,3,0,0,0,10.0,0,110,9.850730e+07,0,0,0,219081.15,9745585.72,21.19
73392,55.01,59.56,14.62,17,1,3,0,0,0,10.0,0,110,9.850730e+07,0,0,0,219081.61,9745585.50,21.60


In [10]:
x, y, z = np.random.random((3, 100))

df['Z'].values
z = df['Z'].values.reshape(-1, 1)

In [11]:
import matplotlib.cm as cm

sm = cm.ScalarMappable(cmap='summer')

col = sm.to_rgba(df['Intensity'].values)

col.shape

(73394, 4)

In [12]:
fig = p3.figure(width=1000)
fig.xlabel='Y'
fig.ylabel='Z'
fig.zlabel='X'
all_points = p3.scatter(df['Y'], df['Z'], df['X'], color=col, size=.2)
p3.squarelim()

p3.show()